In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve
from ast import literal_eval

import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegressionCV, LinearRegression

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, overview, roc_results, model
from lob_data_utils.svm_calculation import lob_svm

sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

/home/vevurka/.conda/envs/py36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
data_length = 10000 # TODO: not used
stock = '9061'

In [3]:
df, df_test = lob.load_prepared_data(
    stock, data_dir='../queue_imbalance/data/prepared', cv=False, include_test=True, length=None)
print(len(df), len(df_test))

13796 3449


In [4]:
def prepare_df(df):
    df.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)
    df.drop(columns=['bid', 'ask'], inplace=True)
    df.index = df['datetime']
    df = df.sort_index()
    df['prev_queue_imbalance'] = [None] + df['queue_imbalance'].iloc[0:len(df)-1].values.tolist()
    df.dropna(inplace=True)
    return df
df = prepare_df(df)
df_test = prepare_df(df_test)
df.head()

,datetime,bid_price,ask_price,mid_price,sum_sell_ask,sum_buy_bid,mid_price_indicator,queue_imbalance,prev_queue_imbalance
datetime,,,,,,,,,
2013-09-16 13:07:00,2013-09-16 13:07:00,705.8,706.1,705.95,7224.0,5045.0,0.0,-0.177602,0.007661
2013-09-16 13:08:00,2013-09-16 13:08:00,705.7,705.9,705.80,582.0,2000.0,0.0,0.549187,-0.177602
2013-09-16 13:09:00,2013-09-16 13:09:00,705.6,705.8,705.70,5031.0,1292.0,0.0,-0.591333,0.549187
2013-09-16 13:10:00,2013-09-16 13:10:00,705.3,705.6,705.45,7042.0,5785.0,0.0,-0.097996,-0.591333
2013-09-16 13:11:00,2013-09-16 13:11:00,704.8,705.0,704.90,8155.0,7191.0,1.0,-0.062818,-0.097996


## Let's try to fit Logistic Regression

In [5]:
df_result = pd.DataFrame()
features = ['queue_imbalance', 'prev_queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)

features = ['queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540..."
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540..."


In [6]:
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100]
coef0s = [0.1, 1, 10, 100]


for g in gammas:
    for c in cs:
        for coef in coef0s:
            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance', 'prev_queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)

            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540...",NaN,NaN,NaN
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540...",NaN,NaN,NaN
2,"[0.0, 0.0, 0.11468531468531466, 0.509369676320...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.008528497021021653, 0.08517698554...","[0.0, 0.0, 0.018314784375449658, 0.08660842473...",svm_sigmoid,"[0.0, 0.0, 0.5466666666666666, 0.5641509433962...","[0.0, 0.0, 0.0640625, 0.4642857142857143, 0.60...","[0.5, 0.5, 0.5043439535830618, 0.5427985948477...","[0.0, 0.0, 0.17527238275698723, 0.524008350730...","[0.0, 0.0, 0.036992110549350254, 0.08926935806...","[0.0, 0.0, 0.0639747647573733, 0.0893105213601...","[0.0, 0.0, 0.5817610062893082, 0.5324565125159...","[0.0, 0.0, 0.10317902955939766, 0.515824085491...","[0.5, 0.5, 0.5178160256989472, 0.5445941952813...",0.1,0.1,0.1
3,"[0.0, 0.0, 0.11235955056179775, 0.506393861892...",queue_imbalance,"[0.0, 0.0, 0.010191235537437726, 0.08045188008...","[0.0, 0.0, 0.02231058736743943, 0.081828228818...",svm_sigmoid,"[0.0, 0.0, 0.5555555555555556, 0.5614366729678...","[0.0, 0.0, 0.0625, 0.4611801242236025, 0.60063...","[0.5, 0.5, 0.5051913680781759, 0.5404261276855...","[0.0, 0.0, 0.16801909307875895, 0.523212045169...","[0.0, 0.0, 0.03548649112190505, 0.089182319792...","[0.0, 0.0, 0.06280386956852303, 0.089232573431...","[0.0, 0.0, 0.5827814569536424, 0.5325670498084...","[0.0, 0.0, 0.09815950920245399, 0.514180024660...","[0.5, 0.5, 0.517083817577357, 0.54454645832348...",0.1,0.1,0.1
4,"[0.0, 0.0, 0.0, 0.24907521578298397, 0.5699168...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.047651945758687586, 0.092665...","[0.0, 0.0, 0.0, 0.07154658588752022, 0.0932652...",svm_sigmoid,"[0.0, 0.0, 0.0, 0.6047904191616766, 0.54088952...","[0.0, 0.0, 0.0, 0.15683229813664595, 0.6022364...","[0.5, 0.5, 0.5, 0.5243177884125853, 0.54634114...","[0.0, 0.0, 0.0, 0.2670886075949367, 0.52843250...","[0.0, 0.0, 0.0, 0.05651938954166902, 0.0866187...","[0.0, 0.0, 0.0, 0.07860534608951192, 0.0866521...","[0.0, 0.0, 0.0, 0.5804676753782668, 0.53612040...","[0.0, 0.0, 0.0, 0.17344841759145088, 0.5209619...","[0.5, 0.5, 0.5, 0.5276843326826786, 0.54328712...",0.1,1.0,0.1
5,"[0.0, 0.0, 0.0, 0.2558425584255843, 0.56518462...",queue_imbalance,"[0.0, 0.0, 0.0, 0.053826871032513046, 0.079918...","[0.0, 0.0, 0.0, 0.08040542794900055, 0.0804957...",svm_sigmoid,"[0.0, 0.0, 0.0, 0.6153846153846154, 0.53495007...","[0.0, 0.0, 0.0, 0.16149068322981366, 0.5990415...","[0.5, 0.5, 0.5, 0.5274666530903167, 0.53996662...","[0.0, 0.0, 0.0, 0.26965582570255764, 0.5270270...","[0.0, 0.0, 0.0, 0.057821859752146154, 0.083750...","[0.0

In [7]:
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100]

for g in gammas:
    for c in cs:
        clf = SVC(kernel='rbf', gamma=g, C=c)
        features = ['queue_imbalance', 'prev_queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)

        clf = SVC(kernel='sigmoid', gamma=g, C=c)
        features = ['queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540...",NaN,NaN,NaN
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540...",NaN,NaN,NaN
2,"[0.0, 0.0, 0.11468531468531466, 0.509369676320...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.008528497021021653, 0.08517698554...","[0.0, 0.0, 0.018314784375449658, 0.08660842473...",svm_sigmoid,"[0.0, 0.0, 0.5466666666666666, 0.5641509433962...","[0.0, 0.0, 0.0640625, 0.4642857142857143, 0.60...","[0.5, 0.5, 0.5043439535830618, 0.5427985948477...","[0.0, 0.0, 0.17527238275698723, 0.524008350730...","[0.0, 0.0, 0.036992110549350254, 0.08926935806...","[0.0, 0.0, 0.0639747647573733, 0.0893105213601...","[0.0, 0.0, 0.5817610062893082, 0.5324565125159...","[0.0, 0.0, 0.10317902955939766, 0.515824085491...","[0.5, 0.5, 0.5178160256989472, 0.5445941952813...",0.1,0.1,0.1
3,"[0.0, 0.0, 0.11235955056179775, 0.506393861892...",queue_imbalance,"[0.0, 0.0, 0.010191235537437726, 0.08045188008...","[0.0, 0.0, 0.02231058736743943, 0.081828228818...",svm_sigmoid,"[0.0, 0.0, 0.5555555555555556, 0.5614366729678...","[0.0, 0.0, 0.0625, 0.4611801242236025, 0.60063...","[0.5, 0.5, 0.5051913680781759, 0.5404261276855...","[0.0, 0.0, 0.16801909307875895, 0.523212045169...","[0.0, 0.0, 0.03548649112190505, 0.089182319792...","[0.0, 0.0, 0.06280386956852303, 0.089232573431...","[0.0, 0.0, 0.5827814569536424, 0.5325670498084...","[0.0, 0.0, 0.09815950920245399, 0.514180024660...","[0.5, 0.5, 0.517083817577357, 0.54454645832348...",0.1,0.1,0.1
4,"[0.0, 0.0, 0.0, 0.24907521578298397, 0.5699168...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.047651945758687586, 0.092665...","[0.0, 0.0, 0.0, 0.07154658588752022, 0.0932652...",svm_sigmoid,"[0.0, 0.0, 0.0, 0.6047904191616766, 0.54088952...","[0.0, 0.0, 0.0, 0.15683229813664595, 0.6022364...","[0.5, 0.5, 0.5, 0.5243177884125853, 0.54634114...","[0.0, 0.0, 0.0, 0.2670886075949367, 0.52843250...","[0.0, 0.0, 0.0, 0.05651938954166902, 0.0866187...","[0.0, 0.0, 0.0, 0.07860534608951192, 0.0866521...","[0.0, 0.0, 0.0, 0.5804676753782668, 0.53612040...","[0.0, 0.0, 0.0, 0.17344841759145088, 0.5209619...","[0.5, 0.5, 0.5, 0.5276843326826786, 0.54328712...",0.1,1.0,0.1
5,"[0.0, 0.0, 0.0, 0.2558425584255843, 0.56518462...",queue_imbalance,"[0.0, 0.0, 0.0, 0.053826871032513046, 0.079918...","[0.0, 0.0, 0.0, 0.08040542794900055, 0.0804957...",svm_sigmoid,"[0.0, 0.0, 0.0, 0.6153846153846154, 0.53495007...","[0.0, 0.0, 0.0, 0.16149068322981366, 0.5990415...","[0.5, 0.5, 0.5, 0.5274666530903167, 0.53996662...","[0.0, 0.0, 0.0, 0.26965582570255764, 0.5270270...","[0.0, 0.0, 0.0, 0.057821859752146154, 0.083750...","[0.0

In [8]:
cs = [0.1, 1, 10, 100]

for c in cs:
    clf = SVC(kernel='linear', C=c)
    features = ['queue_imbalance', 'prev_queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)

    clf = SVC(kernel='sigmoid', gamma=g, C=c)
    features = ['queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540...",NaN,NaN,NaN
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540...",NaN,NaN,NaN
2,"[0.0, 0.0, 0.11468531468531466, 0.509369676320...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.008528497021021653, 0.08517698554...","[0.0, 0.0, 0.018314784375449658, 0.08660842473...",svm_sigmoid,"[0.0, 0.0, 0.5466666666666666, 0.5641509433962...","[0.0, 0.0, 0.0640625, 0.4642857142857143, 0.60...","[0.5, 0.5, 0.5043439535830618, 0.5427985948477...","[0.0, 0.0, 0.17527238275698723, 0.524008350730...","[0.0, 0.0, 0.036992110549350254, 0.08926935806...","[0.0, 0.0, 0.0639747647573733, 0.0893105213601...","[0.0, 0.0, 0.5817610062893082, 0.5324565125159...","[0.0, 0.0, 0.10317902955939766, 0.515824085491...","[0.5, 0.5, 0.5178160256989472, 0.5445941952813...",0.1,0.1,0.1
3,"[0.0, 0.0, 0.11235955056179775, 0.506393861892...",queue_imbalance,"[0.0, 0.0, 0.010191235537437726, 0.08045188008...","[0.0, 0.0, 0.02231058736743943, 0.081828228818...",svm_sigmoid,"[0.0, 0.0, 0.5555555555555556, 0.5614366729678...","[0.0, 0.0, 0.0625, 0.4611801242236025, 0.60063...","[0.5, 0.5, 0.5051913680781759, 0.5404261276855...","[0.0, 0.0, 0.16801909307875895, 0.523212045169...","[0.0, 0.0, 0.03548649112190505, 0.089182319792...","[0.0, 0.0, 0.06280386956852303, 0.089232573431...","[0.0, 0.0, 0.5827814569536424, 0.5325670498084...","[0.0, 0.0, 0.09815950920245399, 0.514180024660...","[0.5, 0.5, 0.517083817577357, 0.54454645832348...",0.1,0.1,0.1
4,"[0.0, 0.0, 0.0, 0.24907521578298397, 0.5699168...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.047651945758687586, 0.092665...","[0.0, 0.0, 0.0, 0.07154658588752022, 0.0932652...",svm_sigmoid,"[0.0, 0.0, 0.0, 0.6047904191616766, 0.54088952...","[0.0, 0.0, 0.0, 0.15683229813664595, 0.6022364...","[0.5, 0.5, 0.5, 0.5243177884125853, 0.54634114...","[0.0, 0.0, 0.0, 0.2670886075949367, 0.52843250...","[0.0, 0.0, 0.0, 0.05651938954166902, 0.0866187...","[0.0, 0.0, 0.0, 0.07860534608951192, 0.0866521...","[0.0, 0.0, 0.0, 0.5804676753782668, 0.53612040...","[0.0, 0.0, 0.0, 0.17344841759145088, 0.5209619...","[0.5, 0.5, 0.5, 0.5276843326826786, 0.54328712...",0.1,1.0,0.1
5,"[0.0, 0.0, 0.0, 0.2558425584255843, 0.56518462...",queue_imbalance,"[0.0, 0.0, 0.0, 0.053826871032513046, 0.079918...","[0.0, 0.0, 0.0, 0.08040542794900055, 0.0804957...",svm_sigmoid,"[0.0, 0.0, 0.0, 0.6153846153846154, 0.53495007...","[0.0, 0.0, 0.0, 0.16149068322981366, 0.5990415...","[0.5, 0.5, 0.5, 0.5274666530903167, 0.53996662...","[0.0, 0.0, 0.0, 0.26965582570255764, 0.5270270...","[0.0, 0.0, 0.0, 0.057821859752146154, 0.083750...","[0.0

In [9]:
df_result['stock'] = [stock] * len(df_result)
df_result.to_csv('res_{}_prev_queue_imbalance.csv'.format(stock))

In [10]:
df_result.sort_values(by='f1', ascending=False).head(5)

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma,stock
87,"[0.5849802371541502, 0.32858707557502737, 0.62...",queue_imbalance,"[0.027023432131060288, 0.018132630304330877, 0...","[0.032045118732543725, 0.02138565840257375, 0....",svm_sigmoid,"[0.4758842443729904, 0.5154639175257731, 0.529...","[0.7589743589743589, 0.24115755627009647, 0.76...","[0.5140163274692422, 0.5090281452236558, 0.528...","[0.5667963683527886, 0.37343096234309625, 0.56...","[-0.030047858186307508, 0.018069371483923846, ...","[-0.03658693285565347, 0.019308345631705217, -...","[0.4571129707112971, 0.4817813765182186, 0.471...","[0.7457337883959044, 0.304867634500427, 0.7071...","[0.4843938403057366, 0.5088287312367505, 0.493...",10.0,10.0,10.0,9061
21,"[0.5813953488372092, 0.4777183600713013, 0.465...",queue_imbalance,"[0.04931508885635083, 0.06387868349579862, -0....","[0.05533510243124821, 0.0651275247398041, -0.0...",svm_sigmoid,"[0.4846066134549601, 0.536, 0.5054945054945055...","[0.7264957264957265, 0.43086816720257237, 0.43...","[0.5254302249817945, 0.5318897798038179, 0.495...","[0.6006779661016949, 0.45251641137855575, 0.47...","[0.08559168645433812, -0.005034287998687992, 0...","[0.0969928903531578, -0.005039574746590001, 0....","[0.4983127109111361, 0.46409335727109513, 0.48...","[0.7559726962457338, 0.44150298889837747, 0.47...","[0.5441540127935256, 0.49749046228763305, 0.50...",10.0,1.0,0.1,9061
37,"[0.5371517027863777, 0.4746666666666666, 0.463...",queue_imbalance,"[0.054334420374840264, 0.055927380009120786, -...","[0.055374755218451484, 0.05696608866753304, -0...",svm_sigmoid,"[0.4908062234794908, 0.5308151093439364, 0.5, ...","[0.5931623931623932, 0.4292604501607717, 0.432...","[0.5275229006170711, 0.5279213643208922, 0.490...","[0.5397553516819572, 0.4586597040905135, 0.478...","[0.04929173047473201, 0.0012797900094431158, 0...","[0.050469542912743644, 0.0012805892318712346, ...","[0.4889196675900277, 0.4676131322094055, 0.486...","[0.6023890784982935, 0.45004269854824935, 0.47...","[0.5249969344587275, 0.5006384023780888, 0.509...",0.1,1.0,1.0,9061
95,"[0.5363703159441586, 0.444649446494465, 0.5781...",queue_imbalance,"[0.009389389239008783, 0.03785329744279953, 0....","[0.009842274354351439, 0.039155172678589464, 0...",svm_sigmoid,"[0.47036082474226804, 0.5216450216450217, 0.53...","[0.6239316239316239, 0.387459807073955, 0.6328...","[0.5047909240734353, 0.5188881313850788, 0.526...","[0.5305232558139535, 0.4471434997750787, 0.517...","[-0.013081148139625576, 0.009374256789329904, ...","[-0.013806770746424328, 0.00941768037867984, -...","[0.4620253164556962, 0.47243346007604564, 0.47...","[0.6228668941979523, 0.42442356959863364, 0.57...","[0.4933196746438864, 0.5046575589204836, 0.494...",100.0,10.0,10.0,9061
19,"[0.5246671887235709, 0.0, 0.6306068601583112, ...",queue_imbalance,"[0.0385743408637923, 0.0, 0.0994429226555047, ...","[0.039140542436844845, 0.0, 0.1074845588951609...",svm_sigmoid,"[0.4841040462427746, 0.0, 0.545662100456621, 0...","[0.5726495726495726, 0.0, 0.746875, 0.66149068...","[0.5195086428270285, 0.5, 0.5493332654723128, ...","[0.5685358255451713, 0.0, 0.5997322623828648, ...","[0.12292192707807292, 0.0, 0.06504122030385695...","[0.12490110070847694, 0.0, 0.07355279592952604...","[0.5229226361031518, 0.0, 0.4998140572703607, ...","[0.6228668941979523, 0.0, 0.7495817066369214, ...","[0.5621819500929881, 0.5, 0.5332469223890549, ...",10.0,0.1,0.1,9061
